In [1]:
import os
os.chdir('../../..')

In [2]:
import convokit

/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
from convokit import Corpus, download
from convokit import HyperConvo
import numpy as np
import pickle

In [7]:
os.listdir('/sauna/reddit_201810_raw/reddit-corpus-small/reddit-corpus-small')

['users.json',
 'conversations.json',
 'utterances.json',
 'corpus.json',
 'index.json']

In [8]:
# corpus = Corpus(filename=download('reddit-corpus-small'))
corpus = Corpus(filename="/sauna/reddit_201810_raw/reddit-corpus-small/reddit-corpus-small")

## 1. First, generate the 3-way tensor (hypergraph_features x (time within comment thread) x threads in subreddit)

Let's get a sense of how we should normalize the time within threads.

In [9]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]

In [10]:
corpus = corpus.reindex_conversations(top_level_comment_ids)

In [11]:
corpus.print_summary_stats()

Number of Users: 114101
Number of Utterances: 279826
Number of Conversations: 10000


In [29]:
next(corpus.iter_conversations()).id

'e6nztuu'

In [31]:
subreddits = [convo.meta['original_convo_meta']['subreddit'] for convo in corpus.iter_conversations()]

In [34]:
os.chdir('convokit/tensors/data')

In [35]:
with open('subreddits.p', 'wb') as f:
    pickle.dump(subreddits, f)

In [36]:
convo_ids = [convo.id for convo in corpus.iter_conversations()]

In [37]:
with open('thread_ids.p', 'wb') as f:
    pickle.dump(convo_ids, f)

In [12]:
hc_transformers = [HyperConvo(prefix_len=i, feat_name="hyperconvo-{}".format(i)) for i in range(2, 11)]

In [13]:
len(hc_transformers)

9

In [14]:
for idx, hc in enumerate(list(reversed(hc_transformers))):
    print(10-idx)
    hc.transform(corpus)

10


/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:11: RuntimeWarning: invalid value encountered in long_scalars
  "norm.max": lambda l: np.max(l) / np.sum(l),
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:14: RuntimeWarning: invalid value encountered in long_scalars
  "norm.2nd-largest": lambda l: np.partition(l, -2)[-2] / np.sum(l) if len(l) > 1 else np.nan,
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2614: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=0)
/home/caleb/Cornell-Conversationa

9
8
7
6
5
4
3
2


Now we construct the tensor:

In [15]:
tensor = np.zeros((9, 10000, 164))

In [16]:
tensor.shape

(9, 10000, 164)

In [17]:
for convo_idx, convo in enumerate(corpus.iter_conversations()):
    for hyperconvo_idx in range(2, 10+1):
        tensor[hyperconvo_idx-2][convo_idx] = list(convo.meta['hyperconvo-{}'.format(hyperconvo_idx)].values())

In [18]:
# impute NaN = -1
tensor[np.isnan(tensor)] = -1

Save the tensor for future use:

In [24]:
os.listdir('/home/caleb/')

['.bash_logout',
 '.bashrc',
 '.profile',
 '.bash_history',
 '.cache',
 '.config',
 'include',
 'lib',
 'lib64',
 'bin',
 'pyvenv.cfg',
 '.nano',
 'quick.py',
 '.conda',
 '.ipython',
 '.jupyter',
 '.local',
 'miniconda',
 'Cornell-Conversational-Analysis-Toolkit',
 'reddit_processing',
 '.wget-hsts',
 'Python-3.7.4',
 '.ssh',
 '.gitconfig',
 '.convokit',
 'subreddit_motif_means_scaled.csv',
 'subreddit_motif_means_scaled_T.csv',
 'cs6742-fork',
 '.python_history',
 'nltk_data',
 'test.txt']

In [90]:
with open('convokit/tensor/hypergraph_tensor.p', 'wb') as f:
    pickle.dump(tensor, f)